In [16]:
import dolfin as dolf
import numpy as np

In [17]:
np.set_printoptions(precision=3)

In [18]:
# Example for the stiffness matrix C
# f_1 for real numbers and f_2 for complex numbers
# We don't apply boundary conditions

In [19]:
def print_blockform(C):
    C = C.getValues(range(size_M), range(size_M))
    K = np.zeros_like(C)
    for i in range(N+1):
        for j in range(N+1):
            C_ij = C[2*i:2*i+2,2*j:2*j+2]
            K[i, j] = C_ij[0, 0]
            K[i, j+N+1] = C_ij[0, 1]
            K[i+N+1, j] = C_ij[1, 0]
            K[i+N+1, j+N+1] = C_ij[1, 1]
    return K

In [20]:
def f_1():
    
    mesh = dolf.UnitIntervalMesh(1)
    
    V = dolf.FunctionSpace(mesh, 'CG', 1)

    u = dolf.TrialFunction(V)
    v = dolf.TestFunction(V)

    dx = dolf.Measure('dx', domain=mesh)

    bcs = []
    
    c_ = u * v * dx
    dummy = v * dx

    C, b = dolf.assemble_system(c_, dummy, bcs)
    C = dolf.as_backend_type(C).mat()
    
    return C

In [21]:
N = 2
size_M = (N+1)*2
mesh = dolf.UnitIntervalMesh(N)

CG = dolf.FiniteElement('CG', mesh.ufl_cell(), 1)
W = dolf.FunctionSpace(mesh, dolf.MixedElement([CG, CG]))

(u_1, u_2) = dolf.TrialFunction(W)
(v_1, v_2) = dolf.TestFunction(W)

dx = dolf.Measure('dx', domain=mesh)

bcs = []


In [22]:
c_11 = v_1 * u_1 * dx
c_ = c_11
dummy = (v_1 + v_2) * dx
C, b = dolf.assemble_system(c_, dummy, bcs)
C = dolf.as_backend_type(C).mat()
print_blockform(C)

array([[0.167, 0.083, 0.   , 0.   , 0.   , 0.   ],
       [0.083, 0.333, 0.083, 0.   , 0.   , 0.   ],
       [0.   , 0.083, 0.167, 0.   , 0.   , 0.   ],
       [0.   , 0.   , 0.   , 0.   , 0.   , 0.   ],
       [0.   , 0.   , 0.   , 0.   , 0.   , 0.   ],
       [0.   , 0.   , 0.   , 0.   , 0.   , 0.   ]])

In [23]:
c_22 = v_2 * u_2 * dx
c_ = c_22
dummy = (v_1 + v_2) * dx
C, b = dolf.assemble_system(c_, dummy, bcs)
C = dolf.as_backend_type(C).mat()
print_blockform(C)

array([[0.   , 0.   , 0.   , 0.   , 0.   , 0.   ],
       [0.   , 0.   , 0.   , 0.   , 0.   , 0.   ],
       [0.   , 0.   , 0.   , 0.   , 0.   , 0.   ],
       [0.   , 0.   , 0.   , 0.167, 0.083, 0.   ],
       [0.   , 0.   , 0.   , 0.083, 0.333, 0.083],
       [0.   , 0.   , 0.   , 0.   , 0.083, 0.167]])

In [24]:
c_11 = v_1 * u_1 * dx
c_22 = v_2 * u_2 * dx
c_ = c_11 + c_22
dummy = (v_1 + v_2) * dx
C, b = dolf.assemble_system(c_, dummy, bcs)
C = dolf.as_backend_type(C).mat()
print_blockform(C)

array([[0.167, 0.083, 0.   , 0.   , 0.   , 0.   ],
       [0.083, 0.333, 0.083, 0.   , 0.   , 0.   ],
       [0.   , 0.083, 0.167, 0.   , 0.   , 0.   ],
       [0.   , 0.   , 0.   , 0.167, 0.083, 0.   ],
       [0.   , 0.   , 0.   , 0.083, 0.333, 0.083],
       [0.   , 0.   , 0.   , 0.   , 0.083, 0.167]])

In [25]:
# Can you see the block structure? In case you can't see it, I will remap C into K

In [26]:
#C = C.getValues(range(size_M), range(size_M))


In [35]:
c_21 = -v_2 * u_1 * dx 
c_12 = v_1 * u_2 * dx
c_ = c_12 + c_21
dummy = (v_1 + v_2) * dx
C, b = dolf.assemble_system(c_, dummy, bcs)
C = dolf.as_backend_type(C).mat()
C.getValues(range(6),range(6))

array([[ 0.   ,  0.167,  0.   ,  0.083,  0.   ,  0.   ],
       [-0.167,  0.   , -0.083,  0.   ,  0.   ,  0.   ],
       [ 0.   ,  0.083,  0.   ,  0.333,  0.   ,  0.083],
       [-0.083,  0.   , -0.333,  0.   , -0.083,  0.   ],
       [ 0.   ,  0.   ,  0.   ,  0.083,  0.   ,  0.167],
       [ 0.   ,  0.   , -0.083,  0.   , -0.167,  0.   ]])

In [36]:
print_blockform(C)

array([[ 0.   ,  0.   ,  0.   ,  0.167,  0.083,  0.   ],
       [ 0.   ,  0.   ,  0.   ,  0.083,  0.333,  0.083],
       [ 0.   ,  0.   ,  0.   ,  0.   ,  0.083,  0.167],
       [-0.167, -0.083,  0.   ,  0.   ,  0.   ,  0.   ],
       [-0.083, -0.333, -0.083,  0.   ,  0.   ,  0.   ],
       [ 0.   , -0.083, -0.167,  0.   ,  0.   ,  0.   ]])

In [37]:
c_ = c_11 + c_12 + c_21 + c_22
C, b = dolf.assemble_system(c_, dummy, bcs)
C = dolf.as_backend_type(C).mat()
C.getValues(range(6),range(6))

Calling FFC just-in-time (JIT) compiler, this may take some time.


array([[ 0.167,  0.167,  0.083,  0.083,  0.   ,  0.   ],
       [-0.167,  0.167, -0.083,  0.083,  0.   ,  0.   ],
       [ 0.083,  0.083,  0.333,  0.333,  0.083,  0.083],
       [-0.083,  0.083, -0.333,  0.333, -0.083,  0.083],
       [ 0.   ,  0.   ,  0.083,  0.083,  0.167,  0.167],
       [ 0.   ,  0.   , -0.083,  0.083, -0.167,  0.167]])

In [28]:
C = C.getValues(range(size_M), range(size_M))
K = np.zeros_like(C)
for i in range(N+1):
    for j in range(N+1):
        C_ij = C[2*i:2*i+2,2*j:2*j+2]
        K[i, j] = C_ij[0, 0]
        K[i, j+N+1] = C_ij[0, 1]
        K[i+N+1, j] = C_ij[1, 0]
        K[i+N+1, j+N+1] = C_ij[1, 1]

In [29]:
K

array([[ 0.   ,  0.   ,  0.   ,  0.167,  0.083,  0.   ],
       [ 0.   ,  0.   ,  0.   ,  0.083,  0.333,  0.083],
       [ 0.   ,  0.   ,  0.   ,  0.   ,  0.083,  0.167],
       [-0.167, -0.083,  0.   ,  0.   ,  0.   ,  0.   ],
       [-0.083, -0.333, -0.083,  0.   ,  0.   ,  0.   ],
       [ 0.   , -0.083, -0.167,  0.   ,  0.   ,  0.   ]])

In [30]:
# Do 'c_11 = v_1 * u_1 * dx' and 'c_22 = v_2 * u_2 * dx' make more sense now?